In [ ]:
pip install soundfile

In [ ]:
pip install librosa

In [ ]:
pip install split_folders

In [ ]:
pip install keras

In [ ]:
conda install git


In [ ]:
pip install git+git://github.com/fchollet/keras.git --upgrade --no-deps

In [ ]:
pip install tensorflow

In [ ]:
pip install tf-nightly

In [ ]:
pip install efficientnet

# Importing the Libraries

In [21]:
import numpy as np
import soundfile as sf
import scipy.signal as ss
import matplotlib.pyplot as plt
import pandas as pd
import sklearn 
import librosa
import IPython.display as ipd
from librosa import core, onset, feature, display
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import copy
import os
import pathlib
from keras.preprocessing.image import ImageDataGenerator
import splitfolders
import keras, keras.layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import Adam, SGD 
import tensorflow as tf 
import csv
from tqdm import tqdm
from tensorflow.keras.applications import EfficientNetB3 as efn

# Loading the Dataset

In [ ]:
dataset = pd.read_csv("birdsong_metadata.csv")
#dataset.head()
x = []
samplerate = []
spectrum = []
spectrogram = []


#Delete the useless collumns of y 
y = dataset.iloc[:, 3]
y = np.array(y)
y = y.reshape(-1, 1)

# Convert the audios to corresponding Spectrograms

In [ ]:
x = []

pathlib.Path('./Birdsongs_Spectrograms').mkdir(parents=True, exist_ok=True)
birds_names = dataset.iloc[:, 3].unique()
for birds_name in birds_names:
  pathlib.Path(f'./Birdsongs_Spectrograms/{birds_name}').mkdir(parents=True, exist_ok=True)  
  for i in range(len(dataset)):
    if os.path.exists(f'./Birdsongs_Spectrograms/{dataset.iloc[i, 3]}'):
      if len(os.listdir(f'./Birdsongs_Spectrograms/{dataset.iloc[i, 3]}')) < 3: 
        x.append(sf.read('songs/xc' + str(dataset['file_id'][i]) + '.flac'))
        samplerate.append(x[i][1])
        plt.specgram(x[i][0], NFFT = 2048, Fs=samplerate[i], Fc=0, noverlap=210, sides='default', mode='default', scale='dB');
        plt.axis('off');
        print(dataset.iloc[i, 3] + ' - ' + str(dataset.iloc[i, 0]))
        plt.savefig('./Birdsongs_Spectrograms/' + dataset.iloc[i, 3] +'/' + str(dataset['file_id'][i]) + '.png')
        plt.clf()
    

# Split the train and test folders and create train and test batches 

In [17]:

splitfolders.ratio('./Birdsongs_Spectrograms/', output = './data/', seed = 1337, ratio=(.8, .2))

image_size = (64, 64) 
path = "./data/"
batch_size = 16

train_data_generator = ImageDataGenerator(width_shift_range = 0.2, height_shift_range = 0.2, 
                                          shear_range = 0.2, zoom_range = 0.1, fill_mode = 'nearest')
train_batches = train_data_generator.flow_from_directory(path + 'train', target_size = image_size, 
                                                         class_mode = 'categorical', shuffle = False, 
                                                         batch_size = batch_size)

test_data_generator = ImageDataGenerator()
test_batches = test_data_generator.flow_from_directory(path + 'val', 
                                                 target_size = image_size, 
                                                 class_mode = 'categorical', shuffle = False, batch_size = batch_size)


Copying files: 264 files [00:00, 596.30 files/s]


Found 176 images belonging to 88 classes.
Found 88 images belonging to 88 classes.


# CNN model with a poor accuracy! 

In [43]:
model = Sequential()
input_shape=(64, 64, 3)
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu')) 
model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(88))
model.add(Activation('softmax'))
model.summary()

epochs = 500
batch_size = 8
learning_rate = 0.01
momentum = 0.9
decay_rate = learning_rate / epochs
optimizer = SGD(lr = learning_rate, momentum = momentum, decay = decay_rate, nesterov = False)
model.compile(optimizer = optimizer, loss ="categorical_crossentropy", metrics = ['accuracy'])

model.fit(train_batches, steps_per_epoch = 50, epochs = 150, validation_data = test_batches, 
                    validation_steps = int(88/batch_size))
model.evaluate_generator(generator = test_batches, steps = 50)

test_batches.reset()
prediction = model.predict_generator(test_batches, steps = 50, verbose = 1)

predicted_class_indices=np.argmax(prediction, axis=1)
labels = (train_batches.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

predictions = predictions[:200]
filenames=test_batches.filenames

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d_21 (Averag (None, 15, 15, 32)        0         
_________________________________________________________________
activation_51 (Activation)   (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_22 (Averag (None, 7, 7, 64)          0         
_________________________________________________________________
activation_52 (Activation)   (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 7, 7, 64)         

C:\Users\Asus\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


C:\Users\Asus\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


50/50 [==============================] - 0s 6ms/step


In [142]:
with open('birdsongs_features.csv', 'w', newline = '') as f:
    writer = csv.writer(f)
    writer.writerow(['Spectral Centroid', 'Spectral Rolloff',  'Spectral Bandwidth', 'Zero-Crossing Rate', 
                    'Tone', 'Chroma SFTF', 'Mel-Frequency Cepstral Coefficients'])
#mfcc = []
    
for i in range(len(dataset)):
  X, s_rate = sf.read('songs/xc' + str(dataset['file_id'][i]) + '.flac')
  mf = librosa.feature.mfcc(y = X, sr = s_rate)
  cf = librosa.feature.chroma_stft(X, s_rate)
  sc = librosa.feature.spectral_centroid(X, s_rate)
  sb = librosa.feature.spectral_bandwidth(X, s_rate)
  sroll = librosa.feature.spectral_rolloff(X, s_rate)
  zcr = librosa.feature.zero_crossing_rate(X)
    
  t = np.mean(librosa.feature.tonnetz(
             y=librosa.effects.harmonic(X),
             sr=s_rate).T,axis=0)
  #to_append = dataset.iloc[i, 3]  
  to_append = f'{np.mean(sc)} {np.mean(sroll)} {np.mean(sb)} {np.mean(zcr)} {t} {np.mean(cf)}'
  for e in mf:
    to_append += f' {np.mean(e)}'
     
  bird_name = dataset.iloc[i, 3]
  to_append += f'{ bird_name}'    
  with open('birdsongs_features.csv', 'a', newline = '') as f:
    writer = csv.writer(f)
    writer.writerow([to_append])

  #print(dataset.iloc[i, 3])

PermissionError: [Errno 13] Permission denied: 'birdsongs_features.csv'

# Loading the new dataset and perform the preprocessing steps

In [ ]:
features_dataset = pd.read_csv('birdsongs_features.csv', error_bad_lines = False)
features_dataset.head()
x = features_dataset.iloc[:, 1:]
y= features_dataset.iloc[:, 0]

#print(x)
#print(y)

le = LabelEncoder()
y = le.fit_transform(y)

for i in range(len(features_dataset)):
  x.iloc[i:, 4] = np.asfarray(x.iloc[i:, 4])
#np.array(x)
#x = x.strip()

sc = StandardScaler()
x = sc.fit_transform(np.asfarray(x))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(88, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

classifier = model.fit(X_train, y_train, epochs=400, batch_size=32)